In [1]:
import sys
sys.path.append("..")
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer
import numpy as np
from tqdm import tqdm, trange
import torch
import torch.nn.functional as F
from retrieval_model import BertEncoder
from torch import nn
import pandas as pd
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
from tqdm import tqdm, trange
import pickle

In [2]:
p_encoder = BertEncoder.from_pretrained("/opt/ml/mrc-level2-nlp-08/retrieval/p_encoder")
q_encoder = BertEncoder.from_pretrained("/opt/ml/mrc-level2-nlp-08/retrieval/q_encoder")
tokenizer =  AutoTokenizer.from_pretrained("klue/bert-base")

In [3]:
def get_relevant_doc(q_encoder, p_encoder, query, context, elastic_score_dict, k=100):
    if torch.cuda.is_available():
        p_encoder.cuda()
        q_encoder.cuda()
    with torch.no_grad():
        p_encoder.eval()
        q_encoder.eval()

        q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
        q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

        p_embs = []
        for p in context:
            p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
            p_emb = p_encoder(**p).to('cpu').numpy()
            p_embs.append(p_emb)

    p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)
    dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
    elastic_score = torch.Tensor([elastic_score_dict[query]])
    temp_score = elastic_score + dot_prod_scores
    # print(dot_prod_scores)
    # print(dot_prod_scores.size())
    # print(temp_score)
    
    #rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
    rank = torch.argsort(temp_score, dim=1, descending=True).squeeze()

    return_context = []
    for i in range(k):
        return_context.append(context[rank[i]])
    
    return dot_prod_scores.squeeze(), return_context #,rank[:k]

In [50]:
with open('/opt/ml/data/elastic_retrieval.bin','rb') as f:
    elastic_valid = pickle.load(f)
with open('/opt/ml/data/elastic_retrieval_score.bin','rb') as f:
    elastic_valid_score = pickle.load(f)

In [5]:
querys = list(elastic_valid.keys())

In [6]:
hybrid = {}
for i in tqdm(range(len(querys))):
    _, re_rank = get_relevant_doc(q_encoder,p_encoder,querys[i],elastic_valid[querys[i]],elastic_valid_score)
    hybrid[querys[i]] = re_rank


100%|██████████| 600/600 [18:22<00:00,  1.84s/it]


In [10]:
with open("/opt/ml/data/hybrid_retrieval_test.bin", "wb") as file:
    pickle.dump(hybrid,file)

In [71]:
idx = 0
print(querys[idx])
print(hybrid[querys[idx]][0])
print()
print(elastic_valid[querys[idx]][0])

유령'은 어느 행성에서 지구로 왔는가?
영화를 시작할 때 해설자는 돌고래가 지구가 철거될 것을 알고 있었다고 설명한다. 돌고래는 사람들에게 뒤공중제비로 지구가 파괴될 것을 알렸지만, 사람들은 이를 잘못 이해하였고 결국 돌고래들은 안녕히, 그리고 물고기는 고마웠어요란 노래를 부르면서, 지구를 떠나게 된다. 그러던 어느 날, 불도저가 집으로 다가오는 소리를 들은 아서 덴트는 자기 집이 우회로를 건설하기 위해서 철거될 것이라는 것을 깨닫게 된다. 아서는 집의 철거를 막기 위해, 그의 집을 철거 하려는 불도저 앞에 드러눕게 된다. 아서의 시도는 그의 친구인 포드 프리펙트 때문에 방해를 받았고, 포드는 아서를 데리고 술집으로 가게 된다. 술집에서 포드는 자기가 길포드에서 온 것이 아니라, 외계인이란 걸 아서에게 말하였다. 포드는 아서가 그에게 보여준 친절(포드가 자동차를 지구를 지배하는 생물이라고 생각하고, 자동차와 악수하려고 할 때, 아서가 그를 길 밖으로 밀어내었다)로 보곤인이 초공간 고속도로를 놓기 위해 지구를 파괴하려는 순간 아서를 구하게 된다. 둘은 겨우 보곤 우주선에 히치하이킹을 하게 되고, 거기서 그들은 함장의 시(보곤의 시는 전우주에서 3번째로 최악이다)를 듣고 난 뒤, 우주로 던져지게 된다. 그들은 순수한 마음호에 의해서 구조되었는데, 순수한 마음호에는 이 우주선을 훔친 은하계 대통령 자포드 비블브락스와, 지금은 트릴리언이라 불리는 트리시아 맥밀런과, 우울증을 앓고 있는 안드로이드 마빈이 타고 있었다. 자포드는 순수한 마음호로 마그라테아에 가서 삶, 우주, 그리고 모든 것에 대한 답을 얻으려고 한다. 그들은 마그라테아로 진로를 돌렸으나, 순수한 마음호에 실린 무한 불가능 확률 추진기는 그들을 비트보들 6행성의 궤도에 올려놓는다. 비트보틀 6행성은 자트라바티드인들과 자포드와 대통령자리를 다투던 후마 카블라의 고향별이다. 후마 카블라는 마그라테아로 가는 좌표가 담겨있는 붉은 육면체를 가지고, 자포드에게 깊은 생각근처에 있는 모든 관점 총을 가져다 준다고 하

In [ ]:
elastic_valid[]